In [2]:
from langchain_groq import ChatGroq

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()
llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv('GROQ_API_KEY'),
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/artificial-intelligence-data-machine-learning-engineering-undergraduate-internship/job/R-68770")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.






















Artificial Intelligence, Data & Machine Learning Engineering Undergraduate Internship










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers








In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Artificial Intelligence, Data & Machine Learning Engineering Undergraduate Internship',
 'experience': '0-2 years',
 'skills': ['Python',
  'SQL',
  'Tableau',
  'AI/ML techniques such as neural networks, tree ensembles, regressions',
  'Data structures and algorithms',
  'Solution and technical design',
  'Problem solving, debugging, testing, and analysis'],
 'description': "Contribute to high-priority, real business initiatives that align with the NIKE, Inc. strategic roadmap, including integrating Machine Learning within Nike's technologies, scaling innovation teams, and ensuring equitable access to Nike's Technology using Artificial Intelligence."}

In [7]:
type(json_res)

dict

In [8]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [11]:
job = json_res
job['skills']

['Python',
 'SQL',
 'Tableau',
 'AI/ML techniques such as neural networks, tree ensembles, regressions',
 'Data structures and algorithms',
 'Solution and technical design',
 'Problem solving, debugging, testing, and analysis']

In [12]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [13]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Sidhartha Sekhar Das, final year B.Tech student at NIT Rourkela, applying for a job role mentioned above. 
        Your job is to write a professional cold email addressed to the hiring manager regarding this opportunity.  
        The email should introduce yourself, highlight your skills and experience relevant to the role, and explain why you are a strong fit.  
        Also mention that you have attached your resume for their consideration. {link_list}  
        Do not provide a preamble.  

        ### EMAIL (NO PREAMBLE):

        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Artificial Intelligence, Data & Machine Learning Engineering Undergraduate Internship

Dear Hiring Manager,

I am Sidhartha Sekhar Das, a final year B.Tech student at NIT Rourkela, and I am excited to apply for the Artificial Intelligence, Data & Machine Learning Engineering Undergraduate Internship role at NIKE, Inc. With a strong foundation in computer science and a passion for machine learning, I am confident that I would be a valuable addition to your team.

Throughout my academic journey, I have developed a solid understanding of programming languages, including Python, and have worked with SQL, Tableau, and various AI/ML techniques such as neural networks, tree ensembles, and regressions. My experience with data structures and algorithms has enabled me to design and implement efficient solutions to complex problems. I am also proficient in solution and technical design, and have honed my problem-solving, debugging, testing, and analysis skills through var